__CLICK TO ZOOM MANDELBROT FRACTAL INTERACTIVE NAVIGATOR__
- __Left click to zoom In,__  
- __Right click to zoom Out.__  
- Figure is resizable (__bottom right drag__), but depending on your hardware it could get too slow
  - 1000x1000 pixels runs ok, meaning a million calculations (* max_iter) each draw
  - There's a glitch on the dpi getter so resize first -_ONCE_, then navigate later is recommended.

# Speed calculations by using a C compiled function
## Use a binary
 - Called mandelbrot.cpython-37m-x86_64-linux-gnu.so  
 - Compile it by running the setup.py pointing to mandelbrot.pyx file  
Then just import it:

In [1]:
from mandelbrot import mandelbrot

## Or compile it on the fly using cython

## Small speed gains
Disabling both autosave and 'Settings>Save Widget State Automatically' also helps

# Import
Check https://github.com/matplotlib/ipympl for installation, but `!pip install ipympl matplotlib numpy Cython` is a start for troubleshooting.  
Also there's a tk and Qt5 backend versions on the repo on separate py files

In [99]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

## If starting over without reseting the whole kernel

In [100]:
plt.close()

# This is it

In [103]:
def button_press(event):
    global c, coords, pendingDraw
    #print(event.name, 'I', c, pendingDraw, sep=',', end='; ')
    #plt.ioff() <- if your computer is very slow
    ax = event.inaxes 
    if event.button == 1:
        #print('# deal with zoom in')
        left, right, bottom, top, rx, ry = coords[c]
        c+=1
        arm_x=(right-left)/4
        arm_y=(top-bottom)/4
        right  = event.xdata + arm_x
        left   = event.xdata - arm_x
        bottom = event.ydata - arm_y
        top    = event.ydata + arm_y
        coords[c] = [left, right, bottom, top, rx, ry ]
        ax.set_xlim(left,right)
        ax.set_ylim(bottom,top)
        # Here you can choose from drawing directly (A) or show the zooming then redrawing (B)
        # A
        #mandraw(ax, left, right, bottom, top, rx, ry )
        #pendingDraw=False
        # B
        pendingDraw=True
        ft.set_text('zoom In {} times, {:.1E} pixels calculated'.format(c, rx*ry))
    elif event.button == 3:
        #print('# deal with zoom out')
        if c==0:
            #print('Already initial image, not zooming out')
            ft.set_text('Already initial image, not zooming out')
            return
        c-=1
        left, right, bottom, top, rx, ry = coords[c]
        ax.set_xlim(left,right)
        ax.set_ylim(bottom,top)
        ft.set_text('zoom Out to {}'.format(c))
        pendingDraw=False
    else:
        # deal with something that should never happen
        print('wtf!')
    #plt.ion() <- if your computer is very slow
    #print(event.name, 'F', c, pendingDraw, sep=',', end='; ')

def resize(event):
    global coords, pendingDraw
    #print(event.name, 'I', c, pendingDraw, sep=',', end='; ')
    rx, ry = get_ax_wh(fig, ax)
    ft.set_text('Resize w{}, h{}, dpi{}'.format( rx, ry, fig.dpi))
    coords[c][4]=rx
    coords[c][5]=ry
    pendingDraw=True
    #print(event.name, 'F', c, pendingDraw, sep=',', end='; ')

def draw(event):
    global pendingDraw
    #print('pre draw', 'I', c, pendingDraw, sep=',', end='; ')
    if pendingDraw:
        #print(event.name, c, sep=',', end='; ')
        mandraw(ax, *coords[c] )
    #else:
    #   print(event.name, 'avoided')
    #print('post draw', 'F', c, pendingDraw, sep=',', end='; ')

def get_ax_wh(fig,ax):
    bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    width, height = int(bbox.width*fig.dpi), int(bbox.height*fig.dpi)    
    return width, height

def mandraw(ax, left, right, bottom, top, rx, ry ):
    global pendingDraw
    #print('pre mandraw', 'I', c, pendingDraw, sep=',', end='; ')
    dx = (right - left ) / rx
    dy = (top - bottom) / ry
    z = np.zeros((ry,rx), dtype=np.int32)
    mandelbrot( z, left, dx, rx, bottom, dy, ry, maxIter)
    ax.pcolorfast( (left,right), (bottom,top), z, cmap='terrain', vmin=0, vmax=maxIter)
    pendingDraw=False
    #print('post mandraw', 'F', c, pendingDraw, sep=',', end=';\n')
    
# init plot
fig, ax = plt.subplots()
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False

# connect events
ids=dict()
#ids[]=fig.canvas.mpl_connect('',)
ids['draw'] = fig.canvas.mpl_connect('draw_event', draw) 
ids['resize'] = fig.canvas.mpl_connect('resize_event', resize)
ids['button_press'] = fig.canvas.mpl_connect('button_press_event', button_press)

# Initial data
left, right = np.float64((-2.0, 0.66))
bottom, top = np.float64((-1.4, 1.4))
rx, ry = 8, 6
coords=dict()
c=0
coords[c] = [left, right, bottom, top, rx, ry ] #print(c,coords[c])
pendingDraw=False
maxIter=63

# Initial plot
ft = fig.text(0.5,0.9,'w{}, h{}, dpi{}'.format( *get_ax_wh(fig,ax), fig.dpi))
mandraw(ax, left, right, bottom, top, rx, ry )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Not seeing enough colors (mainly inside the fractal)?
Increase the max iteration parameter, beware it also slows down.

In [89]:
from ipywidgets import BoundedIntText
bit = BoundedIntText( value=maxIter, min=1, max=256*2, step=10, description='Upper bound cut off iteration number:',
    disabled=False, style = {'description_width': 'initial'} )
def change_max_iter(change):
    global maxIter, pendingDraw
    if isinstance(change.new,int):
        maxIter = change.new
        pendingDraw=True
        fig.canvas.send_event('draw')
        #print(maxIter, pendingDraw, c, pendingDraw, sep=',', end=';\n')
bit.observe(change_max_iter)
bit

BoundedIntText(value=101, description='Upper bound cut off iteration number:', max=512, min=1, step=10, style=…